In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import csv




In [96]:
csv_file = 'numerai_training_data.csv'


with open(csv_file, 'r') as f:
  save = csv.reader(f, delimiter = ',')
  raw_data = list(save)
print (raw_data[:2])

[['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'target'], ['0.943745447280042', '0.401440492760393', '0.0524591961080846', '0.251755752856205', '0.599198006133435', '0.433078735492905', '0.154291986680751', '0.92162389945005', '0.0896378616524796', '0.117733463237376', '0.250726083709401', '0.0780808964926594', '0.823646376733102', '0.196164109166281', '0.712755076070547', '0.867629792388402', '0.409075454411293', '0.307076282705858', '0.949682259112273', '0.607615748394229', '0.41754724196422', '1']]


In [97]:
data = np.delete(raw_data, (0), axis=0)
print (x[:2])
x = np.delete(data, (-1), axis=1)
# x2 = list(map(int, x1))
print (x[:2])
y= data[:,-1]
print (y[:12])
y1 = list(map(int, y))
y1[:] = [n + 1 for n in y1]
y = np.array(y1)

print (y[:12])
print (x.shape)
print (y.shape)


[['0.9437454473' '0.4014404928' '0.0524591961' '0.2517557529'
  '0.5991980061' '0.4330787355' '0.1542919867' '0.9216238995'
  '0.0896378617' '0.1177334632' '0.2507260837' '0.0780808965'
  '0.8236463767' '0.1961641092' '0.7127550761' '0.8676297924'
  '0.4090754544' '0.3070762827' '0.9496822591' '0.6076157484' '0.417547242'
  '1']
 ['0.9291754209' '0.8632704307' '0.9053059674' '0.6987165571'
  '0.4165031513' '0.7700837889' '0.6261877255' '0.7382251254'
  '0.5686345357' '0.6759885132' '0.8072406497' '0.7002876098'
  '0.8101027239' '0.7843144766' '0.6149787032' '0.5986603421'
  '0.3146251866' '0.9773152008' '0.7116968494' '0.3046933619'
  '0.7682498921' '0']]
[['0.943745447280042' '0.401440492760393' '0.0524591961080846'
  '0.251755752856205' '0.599198006133435' '0.433078735492905'
  '0.154291986680751' '0.92162389945005' '0.0896378616524796'
  '0.117733463237376' '0.250726083709401' '0.0780808964926594'
  '0.823646376733102' '0.196164109166281' '0.712755076070547'
  '0.867629792388402' '0

In [114]:
x_train = x[:int(x.shape[0] * 0.7)]
x_validate = x[int(x.shape[0] * 0.7) + 1: int(x.shape[0] * 0.9)]
x_test = x[int(x.shape[0] * 0.9) + 1:]
print (x_train.shape)
print (x_validate.shape)

y_train = y[:int(y.shape[0] * 0.7)]
y_validate = y[int(y.shape[0] * 0.7) + 1: int(y.shape[0] * 0.9)]
y_test = y[int(y.shape[0] * 0.9) + 1:]
print (y_train.shape)
print (y_validate.shape)

(67424, 21)
(19263, 21)
(67424,)
(19263,)


In [143]:
features = 21
num_labels = 2

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, features)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(x_train, y_train)
valid_dataset, valid_labels = reformat(x_validate, y_validate)
test_dataset, test_labels = reformat(x_test, y_test)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (67424, 21) (67424, 2)
Validation set (19263, 21) (19263, 2)
Test set (9631, 21) (9631, 2)


In [156]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([features, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
 
  keep_prob = tf.placeholder(tf.float32)
  hidden1_drop = tf.nn.dropout(hidden1, keep_prob)  
  
 # Training computation.
  logits = tf.matmul(hidden1_drop, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 5e-4 * tf.nn.l2_loss(weights2) + 5e-4 * tf.nn.l2_loss(weights1)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [120]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [155]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.3 }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
         valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 58.402054
Minibatch accuracy: 43.8%
Validation accuracy: 49.4%
Minibatch loss at step 500: nan
Minibatch accuracy: 52.3%
Validation accuracy: 50.6%
Minibatch loss at step 1000: nan
Minibatch accuracy: 46.1%
Validation accuracy: 50.6%
Minibatch loss at step 1500: nan
Minibatch accuracy: 50.0%
Validation accuracy: 50.6%
Minibatch loss at step 2000: nan
Minibatch accuracy: 55.5%
Validation accuracy: 50.6%
Minibatch loss at step 2500: nan
Minibatch accuracy: 53.9%
Validation accuracy: 50.6%
Minibatch loss at step 3000: nan
Minibatch accuracy: 46.9%
Validation accuracy: 50.6%
Test accuracy: 51.3%
